# ReAct Agent Tutorial

This notebook demonstrates the **ReAct (Reasoning + Acting)** pattern for building AI agents that can reason about problems and take actions to solve them.

## What is ReAct?

ReAct is an agent architecture that combines reasoning and acting in an interleaved manner:

1. **Think**: The agent reasons about what it needs to do
2. **Act**: The agent calls a tool to gather information or perform an action
3. **Observe**: The agent receives the tool's result
4. **Repeat**: The cycle continues until the agent has enough information to answer

```
User Question → Think → Act → Observe → Think → Act → Observe → ... → Final Answer
```

This pattern enables agents to break down complex tasks, gather real-world data, and provide informed responses.

## Setup

First, let's install the required packages and set up our API key.

In [ ]:
# Install miiflow-llm with example dependencies
!pip install -q miiflow-llm[examples] nest_asyncio

# Enable nested async for environments that need it (e.g., some Jupyter kernels)
import nest_asyncio
nest_asyncio.apply()

# Or install separately:
# !pip install -q miiflow-llm yfinance

In [ ]:
import os
from getpass import getpass

# Set your OpenAI API key
# Option 1: Enter it when prompted
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Option 2: Or uncomment and paste directly (not recommended for shared notebooks)
# os.environ["OPENAI_API_KEY"] = "sk-..."

print("API key configured!")

## Define Tools

Tools are functions that the agent can call to interact with the world. We'll create tools that fetch real stock data from Yahoo Finance.

The `@tool` decorator registers a function as a tool the agent can use.

In [ ]:
import yfinance as yf
from miiflow_llm.core.tools import tool


@tool("get_stock_quote", "Get real-time stock quote and key metrics for a symbol")
def get_stock_quote(symbol: str) -> str:
    """Fetch current stock price and basic metrics.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
    """
    try:
        ticker = yf.Ticker(symbol.upper())
        info = ticker.info
        
        if not info or "regularMarketPrice" not in info:
            return f"Unable to fetch data for '{symbol}'. Please verify the ticker symbol."
        
        price = info.get("regularMarketPrice", "N/A")
        prev_close = info.get("regularMarketPreviousClose", "N/A")
        
        if isinstance(price, (int, float)) and isinstance(prev_close, (int, float)):
            change = price - prev_close
            change_pct = (change / prev_close) * 100
            change_str = f"${change:+.2f} ({change_pct:+.2f}%)"
        else:
            change_str = "N/A"
        
        # Format market cap
        market_cap = info.get("marketCap", 0)
        if market_cap >= 1e12:
            cap_str = f"${market_cap/1e12:.2f}T"
        elif market_cap >= 1e9:
            cap_str = f"${market_cap/1e9:.2f}B"
        else:
            cap_str = f"${market_cap:,.0f}"
        
        # Format P/E ratio
        pe_ratio = info.get('trailingPE')
        pe_str = f"{pe_ratio:.2f}" if isinstance(pe_ratio, (int, float)) else "N/A"
        
        # Format 52-week range
        week_low = info.get('fiftyTwoWeekLow')
        week_high = info.get('fiftyTwoWeekHigh')
        week_low_str = f"${week_low:.2f}" if isinstance(week_low, (int, float)) else "N/A"
        week_high_str = f"${week_high:.2f}" if isinstance(week_high, (int, float)) else "N/A"
        
        return f"""Stock Quote for {info.get('shortName', symbol)} ({symbol.upper()}):
- Current Price: ${price:.2f}
- Change: {change_str}
- Market Cap: {cap_str}
- P/E Ratio: {pe_str}
- 52 Week Range: {week_low_str} - {week_high_str}"""
    except Exception as e:
        return f"Error fetching quote for {symbol}: {str(e)}"


@tool("get_stock_history", "Get historical stock price data")
def get_stock_history(symbol: str, period: str = "1mo") -> str:
    """Fetch historical price data.
    
    Args:
        symbol: Stock ticker symbol
        period: Time period - 1d, 5d, 1mo, 3mo, 6mo, 1y, ytd, max
    """
    try:
        ticker = yf.Ticker(symbol.upper())
        hist = ticker.history(period=period)
        
        if hist.empty:
            return f"No historical data for {symbol}"
        
        start_price = hist['Close'].iloc[0]
        end_price = hist['Close'].iloc[-1]
        change = end_price - start_price
        change_pct = (change / start_price) * 100
        
        return f"""Historical Data for {symbol.upper()} ({period}):
- Period: {hist.index[0].strftime('%Y-%m-%d')} to {hist.index[-1].strftime('%Y-%m-%d')}
- Starting Price: ${start_price:.2f}
- Ending Price: ${end_price:.2f}
- Change: ${change:+.2f} ({change_pct:+.2f}%)
- Period High: ${hist['High'].max():.2f}
- Period Low: ${hist['Low'].min():.2f}"""
    except Exception as e:
        return f"Error: {str(e)}"


@tool("calculate", "Evaluate mathematical expressions")
def calculate(expression: str) -> str:
    """Safely evaluate a math expression.
    
    Args:
        expression: A mathematical expression like '2 + 2' or 'sqrt(16)'
    """
    import math
    allowed = {
        "abs": abs, "round": round, "min": min, "max": max,
        "sqrt": math.sqrt, "pow": pow, "pi": math.pi, "e": math.e
    }
    try:
        result = eval(expression, {"__builtins__": {}}, allowed)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


print("Tools defined successfully!")
print("Available tools: get_stock_quote, get_stock_history, calculate")

### Test the Tools

Let's verify our tools work before using them with an agent:

In [ ]:
# Test the stock quote tool
print(get_stock_quote("AAPL"))

## Example 1: Simple Stock Lookup

Let's create our first ReAct agent. This example shows the most basic usage - asking a simple question that requires one tool call.

In [ ]:
from miiflow_llm import LLMClient, Agent, AgentType

# Create an LLM client
client = LLMClient.create("openai", model="gpt-4o-mini")

# Create a ReAct agent
agent = Agent(
    client,
    agent_type=AgentType.REACT,
    max_iterations=5,
    system_prompt="You are a helpful financial assistant. Use tools to answer questions about stocks.",
)

# Add our tools
agent.add_tool(get_stock_quote)
agent.add_tool(get_stock_history)
agent.add_tool(calculate)

print("Agent created with 3 tools!")

In [ ]:
# Ask a simple question
result = await agent.run("What is the current price of Apple stock (AAPL)?")
print(result.data)

## Example 2: Multi-Step Reasoning

ReAct shines when the agent needs to gather multiple pieces of information and reason about them. Let's ask a question that requires multiple tool calls.

In [ ]:
# Multi-step question
result = await agent.run("""
I want to analyze Microsoft (MSFT):
1. What is the current price?
2. How has it performed over the past month?
3. What's the percentage change?
""")

print(result.data)

## Example 3: Stock Comparison

Let's ask the agent to compare two stocks. This requires gathering data about both and then reasoning about the comparison.

In [ ]:
result = await agent.run("""
Compare Apple (AAPL) and Microsoft (MSFT):
- Which has a higher market cap?
- Which has a better P/E ratio?
Give me a brief comparison.
""")

print(result.data)

## Example 4: Calculations with Real Data

The agent can combine real-world data with calculations. Let's see how many shares we could buy with a given amount.

In [ ]:
result = await agent.run("""
If I have $10,000 to invest in NVIDIA (NVDA):
1. What's the current price per share?
2. How many whole shares could I buy?
3. How much money would be left over?
""")

print(result.data)

## Example 5: Watch the Agent Think (Streaming)

One of the most powerful features is watching the agent's reasoning in real-time. Let's stream the agent's thought process.

In [ ]:
from miiflow_llm import RunContext
from miiflow_llm.core.react import ReActEventType

print("Query: What's Tesla's price and weekly performance?\n")
print("=" * 50)

# Create a context for streaming
context = RunContext(deps=None, messages=[])

async for event in agent.stream("What's Tesla's (TSLA) current price and how has it performed this week?", context):
    if event.event_type == ReActEventType.THINKING_CHUNK:
        print(event.data.get("delta", ""), end="", flush=True)
    
    elif event.event_type == ReActEventType.ACTION_PLANNED:
        action = event.data.get("action", "")
        print(f"\n\n[Calling tool: {action}]\n")
    
    elif event.event_type == ReActEventType.OBSERVATION:
        obs = event.data.get("observation", "")[:150]
        print(f"[Tool result: {obs}...]\n")
    
    elif event.event_type == ReActEventType.FINAL_ANSWER:
        print("\n" + "=" * 50)
        print("FINAL ANSWER:")
        print(event.data.get("answer", ""))

## Example 6: Error Handling

What happens when we ask about an invalid stock symbol? The agent should handle errors gracefully.

In [ ]:
result = await agent.run("What is the current price of INVALIDXYZ stock?")
print(result.data)

## Try It Yourself!

Now it's your turn. Try asking the agent different questions about stocks:

In [ ]:
# Try your own query!
your_query = "What is Google's stock price and how does it compare to its 52-week high?"

result = await agent.run(your_query)
print(result.data)

## Key Takeaways

1. **ReAct Pattern**: Think → Act → Observe → Repeat until done

2. **When to use ReAct**:
   - Simple to moderately complex queries
   - Questions requiring real-world data lookup
   - Tasks where the number of steps is unknown upfront

3. **Key parameters**:
   - `agent_type=AgentType.REACT` - enables ReAct mode
   - `max_iterations` - limits reasoning cycles (safety)
   - `system_prompt` - sets agent behavior

4. **Tools**: Functions decorated with `@tool` that the agent can call

## Next Steps

For more complex tasks with multiple independent subtasks, check out the **Plan & Execute** tutorial which shows how to break down complex tasks into structured plans.